In [ ]:
import numpy as np
from astropy.modeling.models import Sersic2D
from astropy.modeling import models, fitting
import astropy.io.fits as fits 
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy import stats
import random

In [2]:
y, x = np.mgrid[:9, :9]
hdu = fits.open('subcube_par_HE0232-0900.fits')
Hb_blr = hdu[5].data
OIII_nr = hdu[2].data
OIII_br = hdu[3].data
hdu.close()

In [3]:
amp_OIII_nr = np.max(OIII_nr)
amp_OIII_br = np.max(OIII_br)
amp_Hb_blr = np.max(Hb_blr)

In [4]:
hdu = fits.open('subcube_par_err_HE0232-0900.fits')
Hb_blr_err = hdu[5].data
OIII_nr_err = hdu[2].data
OIII_br_err = hdu[3].data
hdu.close()

In [5]:
[y0_blr,x0_blr]= ndimage.measurements.maximum_position(Hb_blr)#Find the positions of the maximums of the values of an array at labels.

In [6]:
p_init = Sersic2D(amplitude = amp_Hb_blr, r_eff = 5, n=3, x_0=x0_blr, y_0=y0_blr,ellip=2.5,theta=-0.5)
f = fitting.LevMarLSQFitter()
p = f(p_init, x, y, Hb_blr)

res = Hb_blr - p(x,y)

In [17]:
[amp1, r1, n1, x1, y1,ellip1,theta1] = p.parameters

2.9643044220754087

In [8]:
Monte_Carlo_loops = 10
parameters_MC = np.zeros((len(p.parameters),Monte_Carlo_loops))
for l in range(Monte_Carlo_loops):
        iteration_data = np.random.normal(Hb_blr,Hb_blr_err) 
        p_MC_init = Sersic2D(amplitude = amp1, r_eff = r1, n=n1, x_0=x1, y_0=y1,ellip=ellip1,theta=theta1)
        f = fitting.LevMarLSQFitter()
        p_MC = f(p_MC_init, x, y, iteration_data)
        parameters_MC[:,l]=p_MC.parameters
        parameters_err = np.std(parameters_MC,1)  
        [amp1_err, r1_err, n1_err, x1_err, y1_err,ellip1_err,theta1_err] = parameters_err

In [9]:
column_names={'amp':0,'r':1,'n':2,'x0':3,'y0':4,'ellip':5,'theta':6}
columns=[]
for key in column_names.keys():
    columns.append(fits.Column(name=key,format='E',array=[p.parameters[column_names[key]]]))
    columns.append(fits.Column(name=key+'_err',format='E',array=[parameters_err[column_names[key]]]))
coldefs = fits.ColDefs(columns)
hdu = fits.BinTableHDU.from_columns(coldefs)
hdu.writeto('HE0232-0900_updated_sersic_table',overwrite=True)

Filename: HE0232-0900_updated_sersic_table8.fits


In [18]:
(amp_fix,r_fix,n_fix,x0_fix,y0_fix,ellip_fix,theta_fix) = p.parameters

In [19]:
[y0_br,x0_br]= ndimage.measurements.maximum_position(OIII_br)#Find the positions of the maximums of the values of an array at labels.

In [20]:
p_br_init = Sersic2D(amp_OIII_br,r_fix,n_fix,x0_br,y0_br,ellip_fix,theta_fix,fixed={'r':True,'n':True,'ellip':True,'theta':True})                     
f = fitting.LevMarLSQFitter()
p_br = f(p_br_init, x, y, OIII_br)
#res = OIII_br - p_br(x,y)

ValueError: can only convert an array of size 1 to a Python scalar

In [16]:
p_br_init

<Sersic2D(amplitude=17.121610814030817, r_eff=1.9374201610055561, n=0.6810868981935816, x_0=4.0, y_0=4.0, ellip=2.0253540576636113, theta=-0.5090284829431319)>

In [13]:
[amp2,r2,n2,x2,y2,ellip2,theta2] = p_br.parameters

In [15]:
Monte_Carlo_loops = 10
parameters_MC = np.zeros((len(p_br.parameters),Monte_Carlo_loops))
for l in range(Monte_Carlo_loops):
        iteration_data = np.random.normal(OIII_br,OIII_br_err) 
        p_MC_init = Sersic2D(amplitude=amp2,r_eff=r2,n=n2,x_0=x2,y_0=y2,ellip=ellip2,theta=theta2,fixed={'r':True,'n':True,'ellip':True,'theta':True})
        f = fitting.LevMarLSQFitter()
        p_MC = f(p_MC_init, x, y, iteration_data)
        parameters_MC[:,l]=p_MC.parameters
        parameters_err = np.std(parameters_MC,1)  
        [amp2_err, r2_err, n2_err, x2_err, y2_err,ellip2_err,theta2_err] = parameters_err

In [16]:
column_names={'amp':0,'r':1,'n':2,'x0':3,'y0':4,'ellip':5,'theta':6}
columns=[]
for key in column_names.keys():
    columns.append(fits.Column(name=key,format='E',array=[p_br.parameters[column_names[key]]]))
    columns.append(fits.Column(name=key+'_err',format='E',array=[parameters_err[column_names[key]]]))
coldefs = fits.ColDefs(columns)
hdu = fits.BinTableHDU.from_columns(coldefs)
try:
    hdu.writeto('HE0232-0900_OIII_br_sersic_table.fits',overwrite=True)
except PermissionError:
    pass

In [21]:
hdu = fits.open('HE0232-0900_OIII_br_sersic_table.fits')
central_tab = hdu[1].data
central_column = hdu[1].header
r_fix = central_tab.field('r')[0]
n_fix = central_tab.field('n')[0]
ellip_fix = central_tab.field('ellip')[0]
theta_fix = central_tab.field('theta')[0]
hdu.close()
print(r_fix)

2.42569


In [22]:
[y0_nr,x0_nr]= ndimage.measurements.maximum_position(OIII_nr)

In [23]:
p_nr_init = Sersic2D(amplitude = amp_OIII_nr, r_eff = r_fix, n=n_fix, x_0=x0_nr, y_0=y0_nr,ellip=ellip_fix,theta=theta_fix,fixed={'r':True,'n':True,'ellip':True,'theta':True})                     
f = fitting.LevMarLSQFitter()
p_nr = f(p_nr_init, x, y, OIII_nr)
res = OIII_nr - p_nr(x,y)

ValueError: can only convert an array of size 1 to a Python scalar

In [20]:
[amp3,r3,n3,x3,y3,ellip3,theta3] = p_nr.parameters

In [21]:
Monte_Carlo_loops = 10
parameters_MC = np.zeros((len(p_nr.parameters),Monte_Carlo_loops))
for l in range(Monte_Carlo_loops):
        iteration_data = np.random.normal(OIII_nr,OIII_nr_err) 
        p_MC_init = Sersic2D(amplitude=amp3,r_eff=r3,n=n3,x_0=x3,y_0=y3,ellip=ellip3,theta=theta3,fixed={'r':True,'n':True,'ellip':True,'theta':True})
        f = fitting.LevMarLSQFitter()
        p_MC = f(p_MC_init, x, y, iteration_data)
        parameters_MC[:,l]=p_MC.parameters
        parameters_err = np.std(parameters_MC,1)  
        [amp3_err, r3_err, n3_err, x3_err, y3_err,ellip3_err,theta3_err] = parameters_err

In [22]:
column_names={'amp':0,'r':1,'n':2,'x0':3,'y0':4,'ellip':5,'theta':6}
columns=[]
for key in column_names.keys():
    columns.append(fits.Column(name=key,format='E',array=[p_br.parameters[column_names[key]]]))
    columns.append(fits.Column(name=key+'_err',format='E',array=[parameters_err[column_names[key]]]))
coldefs = fits.ColDefs(columns)
hdu = fits.BinTableHDU.from_columns(coldefs)
try:
    hdu.writeto('HE0232-0900_OIII_nr_sersic_table.fits',overwrite=True)
except PermissionError:
    pass

In [23]:
hdu = fits.open('HE0232-0900_OIII_nr_sersic_table.fits')
central_tab = hdu[1].data
central_column = hdu[1].header
r_fix = central_tab.field('r')[0]
n_fix = central_tab.field('n')[0]
ellip_fix = central_tab.field('ellip')[0]
theta_fix = central_tab.field('theta')[0]
hdu.close()
print(r_fix)

2.42569


In [24]:
hdus=[]
#amp_Hb,amp_OIII5007,amp_OIII5007_br,amp_Hb_br,amp_Hb1,amp_Hb2,amp_Fe5018_1,amp_Fe5018_2,m,c
hdus.append(fits.PrimaryHDU())
hdus.append(fits.ImageHDU(p(x,y),name='Hb_blr'))
hdus.append(fits.ImageHDU(p_br(x,y),name='OIII_br'))
hdus.append(fits.ImageHDU(p_nr(x,y),name='OIII_nr'))
hdu = fits.HDUList(hdus)
try:
    hdu.writeto('flux_map_sersic_model_HE0232-0900.fits',overwrite='True')
except PermissionError:
    pass